# Abstraction Layer for energy levels and transitions

Create an additional abstraction layer for the energy levels and transitions of an ion. The level will hold information about an individual level

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import fiasco
import astropy.constants as const
import astropy.units as u

%matplotlib inline
%load_ext snakeviz

/Users/willbarnes/anaconda/envs/fiasco/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [52]:
class Transitions(object):
    
    def __init__(self, wgfa, elvlc):
        self._wgfa = wgfa
        self._elvlc = elvlc
        
    @property
    def is_twophoton(self):
        """
        True if the transition is a two-photon decay
        """
        return self._wgfa['wavelength'] == 0.*u.angstrom
    
    @property
    def is_observed(self):
        """
        True for transitions that connect two observed energy levels
        """
        return self._wgfa['wavelength'] > 0.*u.angstrom
        
    @property
    def wavelengths(self):
        return np.fabs(self._wgfa['wavelength'])
    
    @property
    def upper_level(self):
        return self._wgfa['upper_level']
    
    @property
    def lower_level(self):
        return self._wgfa['lower_level']
    
    @property
    def delta_energy(self):
        energy = np.where(self._elvlc['E_obs'].value == -1, self._elvlc['E_th'].value, self._elvlc['E_obs'])
        energy = u.Quantity(energy, self._elvlc['E_obs'].unit)
        indices = np.vstack([fiasco.util.vectorize_where(self._elvlc['level'], self.lower_level),
                             fiasco.util.vectorize_where(self._elvlc['level'], self.upper_level)])
        delta_energy = np.diff(energy[indices], axis=0).flatten()
        return delta_energy * const.h.cgs * const.c.cgs
        

In [57]:
foo = fiasco.IonBase('H 1')

In [58]:
foo_transitions = Transitions(foo._wgfa, foo._elvlc)

In [36]:
foo_transitions._wgfa['wavelength'].to(u.m) == 0.*u.angstrom

array([False, False, False, ..., False, False,  True])

In [22]:
foo_transitions.lower_level

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  3,  3,  2,  2,  4,  4,  4,
        3,  2,  3,  2,  4,  4,  4,  3,  2,  3,  2,  4,  4,  4,  6,  5,  6,
        5,  8,  7,  8,  7,  7,  8,  9,  9,  9,  6,  5,  6,  5,  7,  8,  8,
        7,  9,  9,  9,  7,  8, 11, 10, 11, 10, 13, 12, 13, 12, 15, 14, 15,
       14, 16, 16, 16, 15, 14, 12, 13])

In [60]:
foo_transitions.delta_energy.to(u.eV)

<Quantity [10.19881064, 10.19881064, 13.05450121, 13.05449836, 12.74853822,
           12.74853264, 12.08750717, 12.08749378, 10.19885168, 10.1988063 ,
            2.85569491,  2.85569231,  2.85569058,  2.85568772,  2.85565053,
            2.85564954,  2.85564693,  2.54973192,  2.54972758,  2.54972684,
            2.549722  ,  2.5496884 ,  2.54968654,  2.54968146,  1.88870087,
            1.88869653,  1.88868885,  1.88868314,  1.88865996,  1.88865549,
            1.88864347,  0.96700743,  0.96700607,  0.96700483,  0.96700322,
            0.96699503,  0.96699503,  0.96699404,  0.96699404,  0.96699144,
            0.96699119,  0.96699107,  0.96699057,  0.96698958,  0.66104444,
            0.66104307,  0.66103935,  0.66103749,  0.66103291,  0.66103291,
            0.66103105,  0.66103105,  0.66102944,  0.66102844,  0.66102658,
            0.66102596,  0.66102547,  0.30596857,  0.30596808,  0.30596597,
            0.30596523,  0.30596399,  0.30596399,  0.30596299,  0.30596299,
            

In [51]:
a = np.array([0,1,2])
b = np.array([1,2,3])
c = np.array([2,4,6,8])
np.diff(c[np.vstack([a,b])],axis=0).flatten().shape

(3,)

In [29]:
(foo._ip['chianti'] * const.h * const.c).cgs.to(u.eV) * (1 - 1/(6**2))

<Quantity 13.22070043 eV>

In [ ]:
class Transition(object):
    
    def __init__(self, lower_level, upper_level, ion_name):
        self._ionbase = fiasco.IonBase(ion_name)
        self._lower_level = lower_level
        self._upper_level = upper_level
        
    def __repr__(self):
        pass
    
    @property
    def _index(self):
        index = np.where(np.logical_and(
            self._ionbase._wgfa['lower_level'] == self._lower_level,
            self._ionbase._wgfa['upper_level'] == self._upper_level))[0]
        if index.shape[0] == 0:
            raise IndexError(f'Transition {upper_level}->{lower_level} not available')
        return index[0]
    
    @property
    def lower_level(self):
        index = np.where(self._ionbase._wgfa['lower_level'][self._index] == self._ionbase._elvlc['level'])
        return Level(index[0][0], self._ionbase._elvlc)
    
    @property
    def upper_level(self):
        index = np.where(self._ionbase._wgfa['upper_level'][self._index] == self._ionbase._elvlc['level'])
        return Level(index[0][0], self._ionbase._elvlc)

    @property
    def wavelength(self):
        return self._ionbase._wgfa['wavelength'][self._index]
    
    @property
    def delta_energy(self):
        return self.upper_level.energy - self.lower_level.energy
    

class Level(object):
    
    def __init__(self, index, elvlc):
        self._index = index
        self._elvlc = elvlc
        
    def __repr__(self):
        return f"""Level: {self.level}
Configuration: {self.configuration}
Orbital Angular Momentum: {self.orbital_angular_momentum_label}
Energy: {self.energy}"""
    
    @property
    def level(self):
        return self._elvlc['level'][self._index]
        
    @property
    def configuration(self):
        return self._elvlc['config'][self._index]
    
    @property
    def multiplicity(self):
        return self._elvlc['mult'][self._index]
    
    @property
    def total_angular_momentum(self):
        return self._elvlc['J'][self._index]
    
    @property
    def orbital_angular_momentum_label(self):
        return self._elvlc['L_label'][self._index]
    
    @property
    def energy(self):
        if self._elvlc['E_obs'][self._index] < 0:
            return (self._elvlc['E_th'][self._index]*const.h.cgs*const.c.cgs).decompose().cgs
        else:
            return (self._elvlc['E_obs'][self._index]*const.h.cgs*const.c.cgs).decompose().cgs

In [ ]:
class TestIon(fiasco.Ion):
    
    def __getitem__(self, key):
        _ = self._elvlc['level'][key]
        return Level(key, self._elvlc)
    
    @property
    def transitions(self):
        transitions = np.stack([self._wgfa['lower_level'], self._wgfa['upper_level']], axis=1)
        return [Transition(*t, self.ion_name) for t in transitions]

In [ ]:
H_ion.__class__.__bases__[0].__bases__[0]

In [ ]:
H_ion = TestIon('h 1', np.logspace(5,7,10)*u.K)

In [ ]:
H_ion._elvlc['level'].shape

In [ ]:
for t in H_ion.transitions[::10]:
    print(t.wavelength)

In [ ]:
H_ion[1]

In [ ]:
H_ion[0]

In [ ]:
%%snakeviz
ts = H_ion.transitions

In [ ]:
H_ion._wgfa['wavelength']

In [ ]:
for level in H_ion:
    print(level.level, level.configuration)

In [ ]:
H_ion._elvlc

In [ ]:
for i,J,L,c in zip(H_ion._elvlc['level'],H_ion._elvlc['J'],H_ion._elvlc['L_label'],H_ion._elvlc['config']):
    print(f'{i}: {L},{J} {c}')

In [ ]:
%%timeit
H_ion[H_ion._scups['upper_level']-1].multiplicity

In [ ]:
%%timeit
H_ion._elvlc['mult'][H_ion._scups['upper_level']-1]

In [ ]:
for level in H_ion:
    print(level.multiplicity)

In [ ]:
H_ion._wgfa

In [ ]:
foo = fiasco.IonBase('Fe 5')

In [ ]:
foo._elvlc['E_obs']

In [ ]:
foo._wgfa['upper_level']

In [ ]:
np.where(np.logical_and(foo._wgfa['lower_level'] == 1, foo._wgfa['upper_level'] == 2))

In [ ]:
np.where(foo._elvlc['level'] == 0)[0].shape[0]

In [ ]:
foo._wgfa['upper_label']

In [ ]:
foobar = TestIon('Fe 11')

In [ ]:
%%snakeviz
fb_t = foobar.transitions

In [ ]:
%%timeit
np.where((foobar._wgfa['lower_level'] == 1) * (foobar._wgfa['upper_level'] == 2))

In [ ]:
%%timeit
np.where(np.logical_and(foobar._wgfa['lower_level'] == 1, foobar._wgfa['upper_level'] == 2))

In [ ]:
foobar[0]

In [7]:
test = fiasco.Ion('H 1', [1e6,1e8]*u.K)

In [8]:
test

CHIANTI Database Ion
---------------------
Name: H 1
Element: hydrogen (1)
Charge: +0
Number of Levels: 25
Number of Transitions: 76

Temperature range: [1.0 MK, 100.0 MK]

HDF5 Database: /Users/willbarnes/.fiasco/chianti_dbase.h5
Using Datasets:
  ioneq: chianti
  abundance: sun_photospheric_1998_grevesse
  ip: chianti

In [9]:
len(test)

TypeError: object of type 'Ion' has no len()

In [4]:
for l in test:
    print(l)

In [16]:
hasattr(test[0], 'level')

True

In [13]:
for level in test:
    print(level,'\n')

Level: 1
Configuration: 1s
Orbital Angular Momentum: S
Energy: 0.0 erg 

Level: 2
Configuration: 2s
Orbital Angular Momentum: S
Energy: 1.634029596469284e-11 erg 

Level: 3
Configuration: 2p
Orbital Angular Momentum: P
Energy: 1.6340289012132456e-11 erg 

Level: 4
Configuration: 2p
Orbital Angular Momentum: P
Energy: 1.6340361716049618e-11 erg 

Level: 5
Configuration: 3s
Orbital Angular Momentum: S
Energy: 1.9366302125401765e-11 erg 

Level: 6
Configuration: 3p
Orbital Angular Momentum: P
Energy: 1.936629994031136e-11 erg 

Level: 7
Configuration: 3p
Orbital Angular Momentum: P
Energy: 1.9366321393926264e-11 erg 

Level: 8
Configuration: 3d
Orbital Angular Momentum: D
Energy: 1.9366321393926264e-11 erg 

Level: 9
Configuration: 3d
Orbital Angular Momentum: D
Energy: 1.9366328545131233e-11 erg 

Level: 10
Configuration: 4s
Orbital Angular Momentum: S
Energy: 2.0425401738999235e-11 erg 

Level: 11
Configuration: 4p
Orbital Angular Momentum: P
Energy: 2.0425400944420903e-11 erg 

Level: 